In [1]:
import numpy as np

import sys
# sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')


from mass_spring import *

import os, sys
import numpy as np
from time import sleep
from pythreejs import *


In [2]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

f0 = mss.add(Fix((0.0, 0.0, 0.0)))

r, z = 0.4, -1.0
p1 = np.array([ r, 0.0, z])
p2 = np.array([-0.5*r,  np.sqrt(3)/2*r, z])
p3 = np.array([-0.5*r, -np.sqrt(3)/2*r, z])

center = (p1 + p2 + p3) / 3.0
omega  = np.array([0.0, 0.0, 40.0])  # tune 20..120

def vrot(p):
    return tuple(np.cross(omega, (p - center)))

def dist(a, b):
    return float(np.linalg.norm(a - b))

In [3]:
m1 = mss.add(Mass(1.0, tuple(p1), vrot(p1)))
m2 = mss.add(Mass(1.0, tuple(p2), vrot(p2)))
m3 = mss.add(Mass(1.0, tuple(p3), vrot(p3)))

# rigid triangle
mss.add(DistanceConstraint(m1, m2, dist(p1, p2)))
mss.add(DistanceConstraint(m2, m3, dist(p2, p3)))
mss.add(DistanceConstraint(m3, m1, dist(p3, p1)))

# pivot 
mss.add(DistanceConstraint(f0, m1, dist(p1, np.array([0.0, 0.0, 0.0]))))


3

In [4]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(
            SphereBufferGeometry(0.15, 16, 16),
            MeshStandardMaterial(color='red'),
            position=(m.pos[0], m.pos[1], m.pos[2])
        )
    )

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(
            SphereBufferGeometry(0.18, 16, 16),
            MeshStandardMaterial(color='blue'),
            position=(f.pos[0], f.pos[1], f.pos[2])
        )
    )

def get_joint_segments():
    segs = []
    for j in mss.joints:
        pA = mss[j.connectors[0]].pos
        pB = mss[j.connectors[1]].pos
        segs.append([[pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]]])
    return segs

joints = None
joint_segments = get_joint_segments()
if joint_segments:
    jointgeo = LineSegmentsGeometry(positions=joint_segments)
    jointmat = LineMaterial(linewidth=4, color='green')
    joints = LineSegments2(jointgeo, jointmat)

axes = AxesHelper(1)


In [5]:
view_width = 1000
view_height = 800

camera = PerspectiveCamera(position=[3, 3, 2], aspect=view_width / view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight(intensity=0.6)

children = [*masses, *fixes, axes, camera, key_light, ambient_light]
if joints is not None:
    children.append(joints)

scene = Scene(children=children)
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer

Renderer(camera=PerspectiveCamera(aspect=1.25, position=(3.0, 3.0, 2.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0,…

In [ ]:
for i in range(5000):
    mss.simulate(0.01, 50)

    for m, mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    if joints is not None:
        joints.geometry = LineSegmentsGeometry(positions=get_joint_segments())

    sleep(0.01)
